In [ ]:
import boto3
import json
import IPython
from pprint import pprint
import time
import urllib.request as request
from IPython.display import HTML, Audio, display, Markdown
import random

def generate_job_name():
    hash = random.getrandbits(32)
    return 'test_transcription_{:6x}'.format(hash)    

client = boto3.client('transcribe', region_name='us-east-1')

In [ ]:
?client.start_transcription_job()

In [ ]:
s3_uri='https://s3.amazonaws.com/shirkeys-aws-summit-us-east-1/my_voice_is_my_passport.mp3'
Audio(s3_uri)

In [ ]:
job_name=generate_job_name()
response = client.start_transcription_job(
    TranscriptionJobName=job_name,
    LanguageCode='en-US',
    MediaFormat='mp3',
    Media={
        'MediaFileUri': s3_uri
    }
)
pprint(response)

In [ ]:
job_name=generate_job_name()
response = client.start_transcription_job(
    TranscriptionJobName=job_name,
    LanguageCode='en-US',
    MediaFormat='mp3',
    Media={
        'MediaFileUri': s3_uri
    }
)
pprint(response)

In [ ]:
while True:
    status = client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Waiting...",end='\r')
    time.sleep(5)
pprint(status)

In [ ]:
url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
pprint(url)
transcript='transcript_{}.txt'.format(job_name)
request.urlretrieve(url,transcript)

In [ ]:
result = json.load(open(transcript))
pprint(result)

In [ ]:
transcript_text = result['results']['transcripts'][0]['transcript']
display(Markdown('### {}'.format(transcript_text)))

In [ ]:
s3_video_uri='https://s3.amazonaws.com/shirkeys-sf-loft-2018/build_intelligent_applications_with_AWS_ML_Services/aws_what_excites.mp4'
s3_uri='https://s3.amazonaws.com/shirkeys-sf-loft-2018/build_intelligent_applications_with_AWS_ML_Services/aws_what_excites.mp3'

HTML('<video controls width="640" height="480" src="{}" />'.format(s3_video_uri))

In [ ]:
Audio(s3_uri)

In [ ]:
job_name=generate_job_name()
response = client.start_transcription_job(
    TranscriptionJobName=job_name,
    LanguageCode='en-US',
    MediaFormat='mp3',
    Media={
        'MediaFileUri': s3_uri
    }
)
pprint(response)

In [ ]:
while True:
    status = client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Waiting...",end='\r')
    time.sleep(5)
pprint(status)
url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
transcript='transcript_{}.txt'.format(job_name)
request.urlretrieve(url,transcript)
result = json.load(open(transcript))
transcript_text = result['results']['transcripts'][0]['transcript']
display(Markdown('### {}'.format(transcript_text)))